<a href="https://colab.research.google.com/github/stoianmihail/HyperTone/blob/main/HyperTone_Fit-Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# lstm model
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

In [12]:
hypertone_dir = '/content/drive/MyDrive/hypertone/'
model_dir = hypertone_dir + 'model/'
model_file = hypertone_dir + 'model/model'
preprocessing_dir = hypertone_dir + 'preprocessing/'

In [3]:
import os
def get_files(path):
  ret = []
  for root, _, files in os.walk(path, topdown=False):
    for file in files:
      ret.append({'path': os.path.join(root,file), 'data' : None})
  return ret

In [4]:
import os
import pickle
import pandas as pd

def compress(a):
  return a[np.logical_or(np.insert(np.diff(a).astype(bool), 0, True), np.asarray(a, dtype=bool))]

def parse_songs():
  songs = {}
  for file in get_files(preprocessing_dir):
    if '.pkl' not in file['path']:
      continue 
    song_name = os.path.basename(file['path']).replace('.pkl', '')
    x = pd.read_pickle(file['path'])
    compressed = compress(np.asarray(x[1]))
    compressed = compressed[compressed != 0]

    # TODO: this is only for these recordings
    # TODO: Next time, also save the label!
    tone = int(x[2])
    #if tone == 1 or tone == 8:# or tone == 5 or tone == 3:
    songs[song_name] = {'x': compressed, 'y': tone, 'raw' : x[0]}
  return songs

songs = parse_songs()
print(songs['1. 9 Ceea ce eşti mai cinstită']['x'])

[ 1 -1  1 -1  1 -1  1  1  1  1 -1  1  5  1  1  1  1  1 -1 -1 -1 -1 -1  8
  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1 -1 -1 -2 -1  1  1  1  1  1  1
 -1  1 -1  1 -7  1  1  4  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -3
  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1  1 -1 -1 -3 -1 -1
 -1 -1  4  1  1 -1 -1 -1 -1 -1 -2  1  4 -1 -1  7  1  1  1 -1 -1 -1  1 -1
 -1 -1 -1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -4  1  1  1  1
  1  1  4  1  1  1  1 -1 -1 -1  3  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1  1  1  1  1 -5  1  1  1  1  1  1  1  1  1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1  1  1 -1 -1 -3 -1 -1 -1 -1  1  1  1
  1  1  1 -1 -1 -1 -1 -1 -1  1 -1]


In [5]:
import tensorflow.keras as keras
kSequenceLength = 128
kNumTones = 8

def update(y):
  return y - 1
  # if y == 1:
  #   return 0
  # return 1

def padarray(A, size):
    t = size - len(A)
    return np.pad(A, pad_width=(0, t), mode='constant')

def generate_training_sequences(seq_len=kSequenceLength):
  # load songs and map them to int
  songs_ = parse_songs()

  inputs, targets = [], []
  for song in songs_:
    x = songs_[song]['x']
    y = songs_[song]['y']
    y = update(y)
    #x[np.insert(np.diff(x).astype(np.bool), 0, True)]
    if len(x) < seq_len:
      x = padarray(x, seq_len)
    num_sequences = len(x) - seq_len + 1
    for index in range(num_sequences):
      inputs.append(np.asarray(x[index : index + seq_len]).reshape((seq_len, 1)))
      targets.append(y)

  # one-hot encode the sequences  
  # inputs size: (# of sequences, sequence length, max(diff))
  #inputs = keras.utils.to_categorical(inputs, num_classes=kOutputUnits)
  inputs = np.asarray(inputs)
  print(inputs.shape)
  #n = len(inputs)
  #inputs = np.reshape(inputs, (n, seq_len, 1))
  # normalize input
  #inputs = inputs / float(kOutputUnits)
  
  targets = keras.utils.to_categorical(targets, num_classes=kNumTones)
  print(targets.shape)
  print(f"There are {len(inputs)} sequences.")
  return inputs, targets

In [6]:
generate_training_sequences()

(53723, 128, 1)
(53723, 8)
There are 53723 sequences.


(array([[[ 1],
         [-1],
         [ 1],
         ...,
         [ 1],
         [ 1],
         [ 1]],
 
        [[-1],
         [ 1],
         [-1],
         ...,
         [ 1],
         [ 1],
         [ 1]],
 
        [[ 1],
         [-1],
         [ 1],
         ...,
         [ 1],
         [ 1],
         [ 1]],
 
        ...,
 
        [[ 1],
         [ 1],
         [-1],
         ...,
         [-1],
         [ 1],
         [ 1]],
 
        [[ 1],
         [-1],
         [-1],
         ...,
         [ 1],
         [ 1],
         [-1]],
 
        [[-1],
         [-1],
         [-1],
         ...,
         [ 1],
         [-1],
         [ 1]]]), array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32))

In [7]:
# load the dataset, returns train and test X and y elements
def load_dataset():
  # load all train
  trainX, trainy = generate_training_sequences()
  print(trainX.shape, trainy.shape)

  # load all test
  testX, testy = generate_training_sequences()
  
  # zero-offset class values
  #trainy = trainy
  #testy = testy
  
  # one hot encode y
  print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  return trainX, trainy, trainX, trainy

load_dataset()

(53723, 128, 1)
(53723, 8)
There are 53723 sequences.
(53723, 128, 1) (53723, 8)
(53723, 128, 1)
(53723, 8)
There are 53723 sequences.
(53723, 128, 1) (53723, 8) (53723, 128, 1) (53723, 8)


(array([[[ 1],
         [-1],
         [ 1],
         ...,
         [ 1],
         [ 1],
         [ 1]],
 
        [[-1],
         [ 1],
         [-1],
         ...,
         [ 1],
         [ 1],
         [ 1]],
 
        [[ 1],
         [-1],
         [ 1],
         ...,
         [ 1],
         [ 1],
         [ 1]],
 
        ...,
 
        [[ 1],
         [ 1],
         [-1],
         ...,
         [-1],
         [ 1],
         [ 1]],
 
        [[ 1],
         [-1],
         [-1],
         ...,
         [ 1],
         [ 1],
         [-1]],
 
        [[-1],
         [-1],
         [-1],
         ...,
         [ 1],
         [-1],
         [ 1]]]), array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32), array([[[ 1],
         [-1],
         [ 1],
         ...,
         [ 1],
       

In [11]:
kNumEpochs = 15
kBatchSize = 64

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, verbose=True):
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  print(n_timesteps, n_features, n_outputs)

  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps, n_features)))
  model.add(Dropout(0.2))
  model.add(BatchNorm())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(BatchNorm())
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()

  filepath = model_dir + "hypertone-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
  checkpoint = ModelCheckpoint(
      filepath,
      monitor='loss',
      verbose=0,
      save_best_only=True,
      mode='min'
  )
  callbacks = [checkpoint]

  # fit network
  model.fit(trainX, trainy, epochs=kNumEpochs, batch_size=kBatchSize, callbacks=callbacks, verbose=verbose)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=kBatchSize, verbose=False)

  # save the model
  import time
  model.save(model_file + '-' + str(int(time.time())) + '.hdf5')
  return accuracy

# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=1):
  # load data
  trainX, trainy, testX, testy = load_dataset()#'/content/drive/MyDrive/hypertone/preprocessing')
  print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

# run the experiment
run_experiment()

(53723, 128, 1)
(53723, 8)
There are 53723 sequences.
(53723, 128, 1) (53723, 8)
(53723, 128, 1)
(53723, 8)
There are 53723 sequences.
(53723, 128, 1) (53723, 8) (53723, 128, 1) (53723, 8)
(53723, 128, 1) (53723, 8) (53723, 128, 1) (53723, 8)
128 1 8
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                6464      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_

NameError: ignored

In [17]:
import librosa
import librosa.display
import IPython.display as ipd

# TODO: this should come into a separate script!
kBinsPerOctave = 24

# Corresponds to a jump from Pa -> Ga
kJump = int(4/3 * kBinsPerOctave)

fmin = librosa.midi_to_hz(36) # C2 (Ison)
fmax = librosa.midi_to_hz(84) # C6 (highest note ever seen in byzantine sheets: A5#)
print(fmin)
print(fmax)

import math

def find_bin(f):
# Find which bin `f` fits into.
# The bins are geometrically distributed.
# Formula: 2**(i/`kBinsPerOctave`) * fmin
# Source: https://en.wikipedia.org/wiki/Constant-Q_transform
  return int(round(math.log2(f / fmin) * kBinsPerOctave)) if not math.isnan(f) else f

def normalize_diffs(bins):
  pitch_diff = []
  last_index_non_nan = None
  for index, elem in enumerate(bins):
    # First position?
    if index == 0:
      continue
    # NaN?
    if math.isnan(elem):
      continue
    # No prev elem which is not NaN?
    if last_index_non_nan is None:
      last_index_non_nan = index
      continue
    diff = bins[index] - bins[last_index_non_nan]

    if abs(diff) > kJump:
      last_index_non_nan = index
      continue

    # isPositive = diff > 0
    # while abs(diff) > kJump and diff < 0:
    #   # `bins[last_index_non_nan]` >= `elem`.
    #   bins[index] += kBinsPerOctave
    #   diff = bins[index] - bins[last_index_non_nan]
      
    # while abs(diff) > kJump and diff > 0:
    #   # `bins[last_index_non_nan]` <= `elem`.
    #   pyin_bins[last_index_non_nan] += kBinsPerOctave
    #   diff = bins[index] - bins[last_index_non_nan]
    
    # Update
    last_index_non_nan = index
    pitch_diff.append(diff)
  return pitch_diff

from scipy.signal import savgol_filter

def solve_audio(filepath_, tone_=0):
  y_, sr_ = librosa.load(filepath_)
  f0_, voiced_flag_, voiced_probs_ = librosa.pyin(y_, fmin=fmin, fmax=fmax)

  # Number of seconds
  num_seconds_ = len(y_) / sr_

  # How many pyin samples / second
  samples_per_second_ = (len(f0_) / num_seconds_)
  
  # Compute the window length
  window_length_ = int(samples_per_second_ / 3)
  if window_length_ % 2 == 0:
    window_length_ += 1

  print(f"samples_per_second={samples_per_second_}, window_length={window_length_}")

  # Filter
  num_skips_ = int(samples_per_second_ * 5)
  yhat_ = savgol_filter(f0_[num_skips_:], window_length_, 1)

  # And determine the bins
  filtered_bins_ = list(map(find_bin, yhat_))
  return [filtered_bins_, normalize_diffs(filtered_bins_), tone_]

65.40639132514966
1046.5022612023945


In [44]:
class HyperTone:
  def __init__(self, model_path_):
    self.model = keras.models.load_model(model_path_)
  
  def extract_tones(self, filepath_, seq_len_=kSequenceLength):
    ret = solve_audio(filepath_)
    x = np.asarray(ret[1])
    x = x[x != 0]

    if len(x) < seq_len_:
      x = padarray(x, seq_len_)
    num_sequences = len(x) - seq_len_ + 1
    seeds = []
    for index in range(num_sequences):
      seeds.append(np.asarray(x[index : index + seq_len_]).reshape((seq_len_, 1)))
    output = self.model.predict(np.asarray(seeds))

    y = []
    for index, elem in enumerate(output):
      y.append(1 + self.sample(elem))
    return y
    #   # make a prediction
    #   #output = self.model.predict(seed)
    #   output = self.model.predict(seed)[0]
    #   #classes_x=np.argmax(predict_x,axis=1)
    #   #print(f"predict_x={predict_x}")
    #   #print(f"classes_x={classes_x}")
    #   y.append(output)
    #   #print(pr

    #   #print(probabilities)
    #   #index = self.sample(probabilities)
    #   #y.append(1 + index)              
    # return y

  def sample(self, probabilites, temperature=0.1):
    """Samples an index from a probability array reapplying softmax using temperature

    :param predictions (nd.array): Array containing probabilities for each of the possible outputs.
    :param temperature (float): Float in interval [0, 1]. Numbers closer to 0 make the model more deterministic.
        A number closer to 1 makes the generation more unpredictable.

    :return index (int): Selected output symbol
    """
    predictions = np.log(probabilites) / temperature
    probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

    choices = range(len(probabilites)) # [0, 1, 2, 3]
    index = np.random.choice(choices, p=probabilites)

    return index

In [47]:
test_filename = '/content/drive/MyDrive/psaltica_nepsis_ch/utrenia_8_glasuri_ierom_macarie/utrenia-glas4-macarie/Inregistrari/4. 15 Preabinecuvântată eşti.m4a'
ht = HyperTone(model_dir + 'hypertone-weights-improvement-15-0.1191.hdf5')
print(ht.extract_tones(test_filename))

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


samples_per_second=43.081287523756046, window_length=15
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
